In [2]:
import filter_junctions
import intron_comparison
import intron_sequences
path = '/home/rozycka/sequences_euglena/euglena_statystyki/files/'

## RNA-seq data has been aligned to *Euglena longa* genome with the help of two programs - HISAT2 and STAR. Then the introns were extracted using RegTools.

### First let's have a look at basic statistics:

In [14]:
file_in = path + 'junctions_longa_regtools.bed'
file_out = path + 'all_introns_longa.bed'
filter_junctions.junctions_to_introns(file_in, file_out)
filter_junctions.intron_stats(file_out)
print('\n')

Number of introns:0 198381
Mean support:  51.14964638750687
Median support:  8




### Because of errors in data and in alignments, there are multiple introns overlapping. We want to choose only one introns in every position that has the strongest support.

In [15]:
cutoff = 1 # First we look at all the best introns

file_in = file_out
file_out = path + 'best_introns_longa.bed'
all_i, best_i = filter_junctions.choose_best_introns(file_in, file_out, cutoff)
filter_junctions.intron_stats(file_out)

filter_junctions.compare_best_introns(all_i, best_i)

Number of introns:0 120073
Mean support:  74.15710442813955
Median support:  15
Mean share of the best intron:  0.8049653364294393
Share of best introns in all:  0.445877834474774
Mean support of the best intron:  84.50790768948889




### All the stats for HISAT are better, so this is the alignment chosen for the following analyses.

### After trying different cutoffs, threshold 50 was chosen, leaving quite a big set of comperatively reliable introns

In [16]:
cutoff = 50
# file_in = f'data/good_junctions_hisat.bed'
file_out = path + 'best_introns_longa_50.bed'

all_i, best_i = filter_junctions.choose_best_introns(file_in, file_out, cutoff)
filter_junctions.intron_stats(file_out)

filter_junctions.compare_best_introns(all_i, best_i)

Number of introns:0 21844
Mean support:  342.0451382530672
Median support:  95.0
Mean share of the best intron:  0.8750643128721719
Share of best introns in all:  0.6270561749651478
Mean support of the best intron:  464.52655191356894




### Introns were also extracted from transcripts, so we compare the two sets and only choose the introns that are idenctical in both.

In [17]:
intron_comparison.extract_introns_from_gtf('/home/halakuc/longa/important/longa_stringtie_strand_informed.gtf'
, path + 'longa_from_gtf.bed')
other_introns = intron_comparison.intron_dict(path + 'longa_from_gtf.bed')

file = path + 'best_introns_longa_50.bed'
file_out = path + 'longa_50_double_checked.bed'
exact_match, one_side_match, diff_lengths = intron_comparison.compare_introns(file, other_introns, file_out)

print('exact match: ', len(exact_match))
# print(exact_match)

print('one side matching: ', len(one_side_match))
# print(one_side_match)

print('length difference distribution:')
print([(x, diff_lengths.count(x)) for x in range(10)])
best_introns = [i[0] for i in exact_match]
genes = set([i.gene for i in best_introns])
print('Number of genes: ', len(genes))

All my introns:  21844
n of pairs:  26801
no match:  2338
exact match:  18143
one side matching:  999
length difference distribution:
[(0, 0), (1, 236), (2, 69), (3, 96), (4, 30), (5, 8), (6, 6), (7, 2), (8, 6), (9, 3)]
Number of genes:  4793


### Preparing .bed files for extracting the sequences with additional margin (intron sequences + a bit of surrounding exons.

In [18]:
# filter_junctions.introns_for_seq('data/introns_hisat_50_cross_checked.bed', 'data/best_introns_hisat_50+0.bed', 0)
filter_junctions.introns_for_seq(path + 'longa_50_double_checked.bed', path + 'longa_50+3.bed', 3)

### After extracting intron sequences they are classified as: conventional if they have the canonical AG|GT junctions, confirmed nonconventional if they can form secondary structure in specific position or unconrfirmed nonconventional otherwise.

In [19]:
path = '/home/rozycka/sequences_euglena/euglena_statystyki/files/'
introns_seq_file = path + 'longa_introny_50+3.fasta' #file with the sequences

introns = intron_sequences.file_to_seq_introns(introns_seq_file, 3)
conv = []
non_conv = []
rest = []
for intron in introns:
    intron.movable_boundary()
    if intron.check_conventional():
        conv.append(intron)
#         print(intron.sequence)
#         break
    elif intron.check_unconventional():
        non_conv.append(intron)
    else:
        rest.append(intron)

print('conv stats: ')
_ = intron_sequences.seq_statistics(conv)

print('\nnonconv stats: ')
_ = intron_sequences.seq_statistics(non_conv)

print('\nrest: ')
_ = intron_sequences.seq_statistics(rest)

conv stats: 
Number of introns:  10058
Mean length:  881.0056671306422
Mean gc:  0.4916012752465492
Tetranucleotides: 
[('TTTT', 116737), ('GGGG', 116154), ('CCCC', 115095), ('AAAA', 114259), ('TGTG', 103573), ('CACA', 100118), ('TTTG', 86426), ('CAAA', 85879), ('GTGT', 79188), ('ACAC', 76305)]
Top junctions:  [('CTAC', 4791), ('GTAG', 4709), ('CTGC', 285), ('GCAG', 273)]

nonconv stats: 
Number of introns:  4750
Mean length:  667.9473684210526
Mean gc:  0.5097534853647481
Tetranucleotides: 
[('CCCC', 42697), ('GGGG', 40666), ('TTTT', 40485), ('TGTG', 36891), ('AAAA', 36447), ('CACA', 35615), ('TTTG', 30434), ('CAAA', 28616), ('GTGT', 27785), ('ACAC', 26683)]
Top junctions:  [('GCCC', 125), ('GGCC', 109), ('GGGC', 105), ('GCGC', 91), ('GGCT', 74), ('GACC', 74), ('TGGC', 72), ('CAGG', 67), ('CAGC', 65), ('CGGG', 62)]

rest: 
Number of introns:  3335
Mean length:  642.8722638680659
Mean gc:  0.5085731819538246
Tetranucleotides: 
[('GGGG', 29790), ('AAAA', 28373), ('CCCC', 26683), ('TTTT'